# Lever leaderboard and channel shift matrix

In [1]:
import sys  # no installation needed
from pathlib import Path  # no installation needed

ROOT = Path(r"C:\\Users\\quantbase\\Desktop\\ecom_forecast")
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))


Modeling choice: CM lever impacts use accounting identity P&L (sessions * CR * AOV minus returns/discounts, COGS, shipping, fees, ad spend). Channel shift responses use a Ridge CM% fit on channel spends as features for a simple sensitivity proxy.

In [2]:
import pandas as pd  # already in env - no new install
import numpy as np  # already in env - no new install

from src.config import ProjectPaths  # no installation needed
from src.optimize import channel_shift_matrix, compute_lever_sensitivities  # no installation needed
from src.models import fit_ridge_standardized, prepare_features, time_split  # no installation needed


In [4]:
import src.optimize as opt  # no installation needed

print("CWD:", Path().resolve())
print("Loaded weighted COGS%:", opt._load_weighted_cogs_pct())

CWD: C:\Users\quantbase\Desktop\ecom_forecast\notebooks
Loaded weighted COGS%: 0.4152434137861537


In [5]:
paths = ProjectPaths.from_root()
paths.ensure_directories()
opt_dir = paths.outputs_dir / 'optimize'
opt_dir.mkdir(parents=True, exist_ok=True)
driver = pd.read_pickle(paths.outputs_dir / 'drivers' / 'driver_matrix.pkl')
driver = driver.sort_values('Day').reset_index(drop=True)
epsilon = 1e-9
if 'return_rate_gross' not in driver.columns:
    driver['return_rate_gross'] = driver['returns_abs'] / (driver['Gross sales'] + epsilon)
if 'discount_rate_gross' not in driver.columns:
    driver['discount_rate_gross'] = (-driver['Discounts']) / (driver['Gross sales'] + epsilon)


In [6]:
lever_table = compute_lever_sensitivities(driver)
lever_table.to_csv(opt_dir / 'lever_leaderboard.csv', index=False)
lever_table.head(10)


,lever,cm$_impact,cm%_impact,new_cm$,new_cm%,base_cm$,base_cm%
0,conversion_up_10bps,438.249648,0.020890,1417.312207,0.075958,979.062559,0.055068
1,cogs_pct_down_1pp,177.793064,0.010000,1156.855623,0.065068,979.062559,0.055068
2,return_rate_down_1pp,115.349253,0.005783,1094.411812,0.060850,979.062559,0.055068
3,aov_up_1pct,99.520839,0.004997,1078.583398,0.060064,979.062559,0.055068
4,ad_spend_up_1000,-1000.000000,-0.056245,-20.937441,-0.001178,979.062559,0.055068


In [7]:
channels = ['Meta_Spend', 'Google_Spend', 'TikTok_Spend']
shift_df = channel_shift_matrix(driver, channels=channels, shift_amount=1000.0)
shift_df.to_csv(opt_dir / 'channel_shift_matrix.csv', index=False)
shift_df


,from_channel,to_channel,delta_cm_pct,delta_cm$
0,Meta_Spend,Google_Spend,0.500779,14586.924833
1,Meta_Spend,TikTok_Spend,0.114422,3332.947307
2,Google_Spend,Meta_Spend,-0.500779,-14586.924833
3,Google_Spend,TikTok_Spend,-0.386356,-11253.977525
4,TikTok_Spend,Meta_Spend,-0.117516,-3423.077456
5,TikTok_Spend,Google_Spend,0.383262,11163.847377


In [9]:
from pathlib import Path  # no installation needed
import pandas as pd  # already in env — no new install

from src.config import ProjectPaths  # no installation needed
import src.optimize as opt  # no installation needed

# --- Load driver matrix ---
ROOT = Path(r"C:\Users\quantbase\Desktop\ecom_forecast")
P = ProjectPaths.from_root(ROOT)

driver_path = P.outputs_dir / "drivers" / "driver_matrix.pkl"
df = pd.read_pickle(driver_path)

# --- Baseline slice (last 30 days) + baseline row (mean of numeric cols) ---
base_df = opt._baseline_slice(df, days=30)  # same as Phase 08
baseline = base_df.mean(numeric_only=True)

# --- Build the exact base_params dict used in compute_lever_sensitivities() ---
base_params = {
    "Sessions": float(baseline.get("Sessions", 0.0)),
    "Conversion rate": float(baseline.get("Conversion rate", 0.0)),
    "aov_proxy": float(baseline.get("aov_proxy", 0.0)),
    "return_rate_gross": float(baseline.get("return_rate_gross", 0.0)),
    "discount_rate_gross": float(baseline.get("discount_rate_gross", 0.0)),
    "weighted_cogs_pct": float(baseline.get("weighted_cogs_pct", baseline.get("cogs_pct", 0.0))),
    "Ad_Spend": float(baseline.get("Ad_Spend", 0.0)),
    "Meta_Spend": float(baseline.get("Meta_Spend", 0.0)),
    "Google_Spend": float(baseline.get("Google_Spend", 0.0)),
    "TikTok_Spend": float(baseline.get("TikTok_Spend", 0.0)),
    "Email_SMS_Cost": float(baseline.get("Email_SMS_Cost", 0.0)),
}

# --- Ensure weighted_cogs_pct is populated using optimize.py fallback ---
if (base_params["weighted_cogs_pct"] is None) or (base_params["weighted_cogs_pct"] <= 0):
    loaded_cogs = opt._load_weighted_cogs_pct()
    if loaded_cogs is not None:
        base_params["weighted_cogs_pct"] = float(loaded_cogs)

# --- Channel shares (used when applying Ad_Spend deltas) ---
channel_cols = ["Meta_Spend", "Google_Spend", "TikTok_Spend", "Email_SMS_Cost"]
channel_total = sum(base_params[c] for c in channel_cols if base_params[c] > 0)
channel_shares = {c: (base_params[c] / channel_total) if channel_total > 0 else 0.0 for c in channel_cols}

# --- Accounting params inferred inside optimize.py (shipping/fees) + baseline P&L calc ---
acct = opt._derive_accounting_params(base_df)
base_pnl = opt._compute_pnl_row(base_params, acct)

print("=== Baseline params (used for sensitivities) ===")
display(pd.Series(base_params).to_frame("baseline_value"))

print("\n=== Channel shares (used for spend deltas) ===")
display(pd.Series(channel_shares).to_frame("share"))

print("\n=== Accounting params inferred by optimize.py ===")
display(pd.Series(acct).to_frame("value"))

print("\n=== Baseline P&L computed in optimize.py ===")
display(pd.Series(base_pnl).to_frame("value"))


=== Baseline params (used for sensitivities) ===


,baseline_value
Sessions,6383.066667
Conversion rate,0.020207
aov_proxy,159.765661
return_rate_gross,0.110019
discount_rate_gross,0.027202
weighted_cogs_pct,0.415243
Ad_Spend,7876.666667
Meta_Spend,4823.333333
Google_Spend,1900.000000
TikTok_Spend,936.666667



=== Channel shares (used for spend deltas) ===


,share
Meta_Spend,0.612357
Google_Spend,0.241219
TikTok_Spend,0.118917
Email_SMS_Cost,0.027507



=== Accounting params inferred by optimize.py ===


,value
shipping_per_order,8.500
merchant_fee_pct,0.025



=== Baseline P&L computed in optimize.py ===


,value
orders,128.982899
gross_sales,20607.038062
net_sales,17779.306418
cogs,7382.739892
shipping,1096.354640
fees,444.482660
ad_spend,7876.666667
cm$,979.062559
cm%,0.055068
